In [1]:
from NextVisionML import MLContext
from NextVisionML.train.interfaces.VarianceFilter import VarianceFilter

In [2]:
#Create an context (Loads data)
mlcontext = MLContext()

In [3]:

#add  feature selection methods
mlcontext.hooks.append(VarianceFilter(mlcontext)) 
#Add data prepocessing methods

#select a model type

#start the training process
mlcontext.start_train_process()

100%|██████████| 10/10 [00:10<00:00,  1.03s/trial, best loss: 0.6910694885921967]


In [4]:
import pandas as pd
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

,Risk Level
0,low
1,low
2,low
3,low
4,low


In [30]:
train_dataset = pd.concat([mlcontext.iter_train_X[2], mlcontext.iter_train_y[2]], axis = 0)
cols = []#["Amb_WindSpeed_Std", "Avg_Precipitation", "Grd_Prod_CosPhi_Avg", "Max_Precipitation", "Min_Precipitation"]
train_dataset = train_dataset.drop(columns=cols)
d = dice_ml.Data(
    dataframe=train_dataset,
    continuous_features= list(train_dataset.drop(columns=["Risk Level"]).columns),
    outcome_name="Risk Level")

# Using sklearn backend
m = dice_ml.Model(model=mlcontext.iter_objs[2]["model"], backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [32]:
test_data = mlcontext.iter_test_X[2].drop(columns = cols)

In [38]:
e1 = exp.generate_counterfactuals(test_data.iloc[:5].copy(), total_CFs=2, desired_class=0)

  0%|          | 0/5 [00:00<?, ?it/s]c:\Program Files\Python310\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_instance_df[self.data_interface.outcome_name] = \
 20%|██        | 1/5 [00:03<00:13,  3.37s/it]c:\Program Files\Python310\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_instance_df[self.data_interface.outcome_name] = \
 40%|████      | 2/5 [00:06<00:10,  3.35s/it]c:\Program Files\

In [39]:
e1.visualize_as_dataframe(show_only_changes=False)

Query instance (original outcome : low)


,datapoint_id,AirRessureSensorZeroOffset,Amb_Temp_Avg,Amb_WindDir_Abs_Avg,Amb_WindDir_Relative_Avg,Amb_WindSpeed_Avg,Amb_WindSpeed_Est_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Std,...,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_TotReactPwr,Rtr_RPM_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Std,Spin_Temp_Avg,Var_Windspeed1,Var_Windspeed2,Risk Level
0,1000001,600.0,22.0,261.100006,7.8,3.5,3.5,6.2,0.8,0.6,...,-2148.0,-2424.0,3.8,11.1,0.0,3.1,24.0,0.06,0.07,low



Diverse Counterfactual set (new outcome: high)


,datapoint_id,AirRessureSensorZeroOffset,Amb_Temp_Avg,Amb_WindDir_Abs_Avg,Amb_WindDir_Relative_Avg,Amb_WindSpeed_Avg,Amb_WindSpeed_Est_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Std,...,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_TotReactPwr,Rtr_RPM_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Std,Spin_Temp_Avg,Var_Windspeed1,Var_Windspeed2,Risk Level
0,1000001,600.0,22.0,261.1,7.8,3.5,3.5,6.2,0.8,0.6,...,-2148.0,-2424.0,3.8,11.1,0.0,3.1,18.3,0.06,0.07,high
1,1000001,600.0,22.0,261.1,7.8,3.5,3.5,6.2,0.8,0.6,...,-2148.0,-2424.0,3.8,11.1,0.0,3.1,20.7,0.06,0.07,high


Query instance (original outcome : low)


...

,datapoint_id,AirRessureSensorZeroOffset,Amb_Temp_Avg,Amb_WindDir_Abs_Avg,Amb_WindDir_Relative_Avg,Amb_WindSpeed_Avg,Amb_WindSpeed_Est_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Std,...,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_TotReactPwr,Rtr_RPM_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Std,Spin_Temp_Avg,Var_Windspeed1,Var_Windspeed2,Risk Level
0,1000004,600.0,22.0,251.8,-1.5,2.3,2.3,3.7,1.0,0.4,...,-1831.0,-2093.0,1.0,2.1,0.0,0.9,24.0,0.06,0.05,high
1,1000004,600.0,22.0,251.8,-1.5,2.3,2.3,3.7,1.0,0.4,...,-1831.0,-2093.0,1.0,2.1,0.0,0.9,9.9,0.06,0.05,high


Query instance (original outcome : low)


,datapoint_id,AirRessureSensorZeroOffset,Amb_Temp_Avg,Amb_WindDir_Abs_Avg,Amb_WindDir_Relative_Avg,Amb_WindSpeed_Avg,Amb_WindSpeed_Est_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Std,...,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_TotReactPwr,Rtr_RPM_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Std,Spin_Temp_Avg,Var_Windspeed1,Var_Windspeed2,Risk Level
0,1000005,600.0,22.0,251.699997,-1.6,2.0,2.0,3.0,1.0,0.2,...,-1736.0,-2005.0,0.8,1.7,0.0,0.8,24.0,0.17,0.18,low



Diverse Counterfactual set (new outcome: high)


,datapoint_id,AirRessureSensorZeroOffset,Amb_Temp_Avg,Amb_WindDir_Abs_Avg,Amb_WindDir_Relative_Avg,Amb_WindSpeed_Avg,Amb_WindSpeed_Est_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Std,...,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_TotReactPwr,Rtr_RPM_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Std,Spin_Temp_Avg,Var_Windspeed1,Var_Windspeed2,Risk Level
0,1000005,600.0,22.0,251.7,-1.6,2.0,2.0,3.0,1.0,0.2,...,-1736.0,-2005.0,0.8,1.7,0.0,0.8,22.4,0.17,0.18,high
1,1000005,600.0,22.0,251.7,-1.6,2.0,2.0,3.0,1.0,1.7,...,-1736.0,-2005.0,0.8,1.7,0.0,0.8,24.0,0.17,0.18,high
